In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupShuffleSplit, GroupKFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, log_loss
import joblib
import os
from datasplitter import mimic_train_test_split

os.makedirs("../models", exist_ok=True)

In [2]:
# import data
df_cleaned = pd.read_csv("../data/MIMIC-ED/event_level_cleaned_sirs.csv")

# count NaNs
nan_counts = df_cleaned.isna().sum().sum()
print("Number of NaNs in the DataFrame:", nan_counts)

# analyze data types of each column
print(df_cleaned.dtypes)
# find first row with sepsis diagnosis
sepsis_rows = df_cleaned[df_cleaned["sepsis_dx"] == 1]
print(f"Number of sepsis diagnosis rows: {len(sepsis_rows)}")
is_onset_rows = df_cleaned[df_cleaned["is_sepsis_onset"] == 1]
print(f"Number of sepsis onset rows: {len(is_onset_rows)}")
df_cleaned.head()

Number of NaNs in the DataFrame: 4537570
subject_id             int64
stay_id                int64
charttime             object
temperature          float64
heartrate            float64
resprate             float64
o2sat                float64
sbp                  float64
dbp                  float64
pain                 float64
rhythm_flag          float64
med_rn               float64
gsn_rn               float64
gsn                  float64
is_antibiotic          int64
ndc                  float64
etc_rn               float64
etccode              float64
sepsis_dx_any        float64
sepsis_dx              int64
sirs_count             int64
sirs_ge2               int64
sepsis_onset_time     object
is_sepsis_onset        int64
dtype: object
Number of sepsis diagnosis rows: 19729
Number of sepsis onset rows: 105502


,subject_id,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,...,is_antibiotic,ndc,etc_rn,etccode,sepsis_dx_any,sepsis_dx,sirs_count,sirs_ge2,sepsis_onset_time,is_sepsis_onset
0,10000032,32952584,2180-07-22 16:36:00,98.1,83.0,24.0,97.0,90.0,51.0,0.0,...,0,1.093905e+10,1.0,1158.0,0.0,0,1,0,NaN,0
1,10000032,32952584,2180-07-22 16:43:00,98.1,85.0,22.0,98.0,76.0,39.0,0.0,...,0,1.093905e+10,1.0,1158.0,0.0,0,1,0,NaN,0
2,10000032,32952584,2180-07-22 16:45:00,98.1,84.0,22.0,97.0,75.0,39.0,0.0,...,0,1.093905e+10,1.0,1158.0,0.0,0,1,0,NaN,0
3,10000032,32952584,2180-07-22 17:26:00,98.1,84.0,22.0,97.0,75.0,39.0,0.0,...,0,3.535602e+10,1.0,6461.0,0.0,0,1,0,NaN,0
4,10000032,32952584,2180-07-22 17:26:00,98.1,84.0,22.0,97.0,75.0,39.0,0.0,...,0,1.054409e+10,1.0,2656.0,0.0,0,1,0,NaN,0


In [3]:
# group patients by "stay_id" and take 2000 random patients
patient_ids = df_cleaned["stay_id"].unique()
# make sure there are 1000 septic and 1000 nonseptic patients
#extract stay_ids with sepsis_dx_any == 1
septic_patient_ids = df_cleaned[df_cleaned["sepsis_dx_any"] == 1]["stay_id"].unique()
# count of septic patients
count_septic = len(septic_patient_ids)
print("Number of septic patients:", count_septic)
nonseptic_patient_ids = df_cleaned[df_cleaned["sepsis_dx_any"] == 0]["stay_id"].unique()
# take 2 times septic number of random patients from the nonseptic patients
random_nonseptic_patient_ids = np.random.choice(nonseptic_patient_ids, size=count_septic * 2, replace=False)
random_patient_ids = np.concatenate([septic_patient_ids, random_nonseptic_patient_ids])
#shuffle the ids and filter df to only include these patients
np.random.shuffle(random_patient_ids)
df_small = df_cleaned[df_cleaned["stay_id"].isin(random_patient_ids)]
df_small = df_small.drop(columns=["sepsis_dx_any"])
print("Number of patients in small df:", df_small["stay_id"].nunique())

# do gssplit with this small df
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
y = df_small["sepsis_dx"]
X = df_small.drop(columns=["sepsis_dx", "charttime", "is_antibiotic","sirs_count", "sirs_ge2","sepsis_onset_time", "is_sepsis_onset"])
train_idx, test_idx = next(gss.split(X, y, groups=df_small["stay_id"]))
x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# print column names in X
print(X.columns)


Number of septic patients: 2339
Number of patients in small df: 7017
Index(['subject_id', 'stay_id', 'temperature', 'heartrate', 'resprate',
       'o2sat', 'sbp', 'dbp', 'pain', 'rhythm_flag', 'med_rn', 'gsn_rn', 'gsn',
       'ndc', 'etc_rn', 'etccode'],
      dtype='object')


In [4]:
# x_train, x_test, y_train, y_test = mimic_train_test_split()

In [5]:
# random forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
print("BCE:", log_loss(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))

BCE: 5.058711238527753
MAE: 0.1403495695598703
MSE: 0.1403495695598703
R2: -0.07159945403008061


In [6]:
# get type 1 error rate
y_test_binary = (y_test >= 0.5).astype(int)
percentile_50 = np.percentile(y_pred, 50)
y_pred_binary = (y_pred >= percentile_50).astype(int)
false_positives = np.sum((y_test_binary == 0) & (y_pred_binary == 1))
true_negatives = np.sum(y_test_binary == 0)
type_1_error_rate = false_positives / (false_positives + true_negatives)
print("Type 1 Error Rate:", type_1_error_rate) 

# get type 2 error rate
false_negatives = np.sum((y_test_binary == 1) & (y_pred_binary == 0))
true_positives = np.sum(y_test_binary == 1)
type_2_error_rate = false_negatives / (false_negatives + true_positives)
print("Type 2 Error Rate:", type_2_error_rate)

Type 1 Error Rate: 0.5
Type 2 Error Rate: 0.0


In [7]:
# get one decision tree and visualize it
estimator = rf.estimators_[0]
from sklearn.tree import export_graphviz
import graphviz
dot_data = export_graphviz(estimator, out_file=None, 
                           feature_names=x_test.columns,  
                           filled=True, rounded=True,  
                           special_characters=True)
graph = graphviz.Source(dot_data)
graph.render("../models/random_forest_tree")


'../models/random_forest_tree.pdf'

In [ ]:
# --- Targets & groups ---
TARGET = "sepsis_dx"  # binary labels: 0/1

groups = df_small["stay_id"].values

X_train = x_train.copy()
X_test  = x_test.copy()
y_train = y_train.copy()
y_test  = y_test.copy()
groups_train = groups[train_idx]

# --- Imports specific to classification ---
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedGroupKFold
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    accuracy_score,
    f1_score,
    confusion_matrix,
    classification_report
)
import numpy as np
import pandas as pd
import joblib
import os

os.makedirs("../models", exist_ok=True)

# --- Pipeline ---
pipe = Pipeline([
    ("rf", RandomForestClassifier(random_state=42, n_jobs=-1))
])

# --- Stratified Group K-fold CV for grid search (keeps label balance within groups) ---
n_unique = np.unique(groups_train).size
n_splits = min(5, max(2, n_unique))  # at least 2, up to 5
cv = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)

# --- Param grid (classification-centric; includes optional class_weight) ---
param_grid = {
    "rf__n_estimators": [200, 400],
    "rf__max_depth": [None, 12, 24, 36],
    "rf__min_samples_split": [2, 5, 10],
    "rf__min_samples_leaf": [2, 4],
    "rf__max_features": ["sqrt"],  # sqrt, half, or all features
    "rf__bootstrap": [True]
}

# --- Multiple classification metrics; refit by ROC-AUC (probability-based) ---
scoring = {
    "roc_auc": "roc_auc",
    "pr_auc": "average_precision",  # area under PR curve (AP)
    "f1": "f1",
    "accuracy": "accuracy",
}

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=scoring,
    refit="roc_auc",
    cv=cv,
    n_jobs=-1,
    verbose=1,
    return_train_score=False,
)

# --- Run grid search (group-aware, stratified) ---
grid.fit(X_train, y_train, groups=groups_train)

print(f"\nBest params ({n_splits}-fold CV):\n{grid.best_params_}")
print(f"Best CV ROC-AUC: {grid.cv_results_['mean_test_roc_auc'][grid.best_index_]:.4f}")
print(f"Best CV PR-AUC:  {grid.cv_results_['mean_test_pr_auc'][grid.best_index_]:.4f}")
print(f"Best CV F1:      {grid.cv_results_['mean_test_f1'][grid.best_index_]:.4f}")
print(f"Best CV Acc:     {grid.cv_results_['mean_test_accuracy'][grid.best_index_]:.4f}")

# --- Evaluate best model on the held-out test set ---
best_pipe = grid.best_estimator_
y_prob = best_pipe.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.5).astype(int)

test_roc_auc = roc_auc_score(y_test, y_prob)
test_pr_auc  = average_precision_score(y_test, y_prob)
test_acc     = accuracy_score(y_test, y_pred)
test_f1      = f1_score(y_test, y_pred)
cm           = confusion_matrix(y_test, y_pred)

print("\nHeld-out Test Metrics:")
print(f"ROC-AUC: {test_roc_auc:.4f}")
print(f"PR-AUC:  {test_pr_auc:.4f}")
print(f"F1:      {test_f1:.4f}")
print(f"Acc:     {test_acc:.4f}")
print("\nConfusion Matrix [tn, fp; fn, tp]:")
print(cm)
print("\nClassification report:")
print(classification_report(y_test, y_pred, digits=4))

# --- Feature importances (map back to column names if available) ---
rf = best_pipe.named_steps["rf"]
importances = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("\nTop 20 features:\n", importances.head(20))

# --- Save artifacts ---
results_df = pd.DataFrame(grid.cv_results_)
results_df.to_csv("../models/rf_clf_gridsearch_results.csv", index=False)
importances.to_csv("../models/rf_clf_best_feature_importances.csv", header=["importance"])
joblib.dump(best_pipe, "../models/rf_clf_best_grid.pkl")

print("\nArtifacts written:")
print("  Best model:           ../models/rf_clf_best_grid.pkl")
print("  CV results:           ../models/rf_clf_gridsearch_results.csv")
print("  Feature importances:  ../models/rf_clf_best_feature_importances.csv")


Fitting with 5-fold GroupKFold CV on 2 param grids; GPU enabled for XGB/LGBM (if installed)
 - XGBoost is installed and will be used
Fitting 5 folds for each of 360 candidates, totalling 1800 fits


/home/ykim3041/septic6/sepsis/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [3]:
df = pd.read_csv("../data/MIMIC-ED/event_level_cleaned.csv")

# load best model and predict again to verify
best_pipe_loaded = joblib.load("../models/rf_best_grid.pkl")

In [6]:
# get bce loss with this loaded model
x_test = df.drop(columns=["sepsis_dx", "charttime", "is_antibiotic", "sepsis_dx_any", "stay_id", "subject_id"])
y_test = df["sepsis_dx"]
y_pred_loaded = best_pipe_loaded.predict(x_test)
print("BCE with loaded model:", log_loss(y_test, y_pred_loaded))


BCE with loaded model: 0.4517256145296766
